In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import numpy as np
from Module.sam import sam_model_registry, SamPredictor
from Module.sam.utils.transforms import ResizeLongestSide

C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
D:\015_segmentation\Custom_SAM\Module\sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with Module.sam.modeling.tiny_vit_sam.

## Model

In [15]:
model_type = 'vit_t'
checkpoint = "../runs/241119_vit-b_to_vit-t_DANU_WS_v2/tiny_vit_best.pth"

model_type = 'vit_b'
checkpoint = "../runs/241115_SAM_ViT_b_dataset_V2_ft_v1/best.pth"

In [16]:
output_dir = "../Weights/onnx"

output_name = "water_seg_vit_b_encoder.onnx"

output_path = os.path.join(output_dir, output_name)

In [17]:
sam = sam_model_registry[model_type](checkpoint=checkpoint)
transform = ResizeLongestSide(sam.image_encoder.img_size)

pixel_mean = torch.Tensor([123.675, 116.28, 103.53]).view(-1, 1, 1)
pixel_std = torch.Tensor([58.395, 57.12, 57.375]).view(-1, 1, 1)

D:\015_segmentation\Custom_SAM\Module\sam\build_sam.py:154: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [18]:
dummy_input = np.zeros((sam.image_encoder.img_size, sam.image_encoder.img_size, 3), dtype=np.uint8)
dummy_input = transform.apply_image(dummy_input)
dummy_input = torch.as_tensor(dummy_input, device='cpu')
dummy_input = dummy_input.permute(2, 0, 1).contiguous()[None, :, :, :]

#dummy_input = (dummy_input - pixel_mean) / pixel_std

In [19]:
dummy_input.shape

torch.Size([1, 3, 1024, 1024])

In [20]:
class Model(torch.nn.Module):
    def __init__(self, image_size, checkpoint, model_type):
        super().__init__()
        self.sam = sam_model_registry[model_type](checkpoint=checkpoint)
        self.sam.to(device='cpu')
        self.predictor = SamPredictor(self.sam)
        self.image_size = image_size

    def forward(self, x):
        self.predictor.set_torch_image(x, (self.image_size))
        return self.predictor.get_image_embedding()


In [21]:
sam_model =Model(image_size=(1024,1024),
                checkpoint=checkpoint,
                model_type=model_type)

In [22]:
model_trace = torch.jit.trace(sam_model, dummy_input)

D:\015_segmentation\Custom_SAM\Module\sam\predictor.py:81: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  and transformed_image.shape[1] == 3
D:\015_segmentation\Custom_SAM\Module\sam\predictor.py:82: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  and max(*transformed_image.shape[2:]) == self.model.image_encoder.img_size
D:\015_segmentation\Custom_SAM\Module\sam\modeling\image_encoder.py:258: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be 

In [23]:
torch.onnx.export(model_trace, 
                  dummy_input, 
                  output_path,
                  input_names=['image'], 
                  output_names=["image_embeddings"], 
                  dynamic_axes={
                      "image": {0: "batch_size"}
                  },
                  opset_version=20)

C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\torch\onnx\utils.py:884: UserWarning: no signature found for <torch.ScriptMethod object at 0x0000027C918E5940>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


In [24]:
output_path

'../Weights/onnx\\water_seg_vit_b_encoder.onnx'